<a href="https://colab.research.google.com/github/jinyoungkim0214/portfolio/blob/main/2nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
Change directory to where this file is located
"""
%cd /content/drive/MyDrive/Homework 1-1

/content/drive/MyDrive/Homework 1-1


# Import Modules

In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from mnist.data_utils import load_data

#Utils

In [ ]:
def sigmoid(z):
    """
    Do NOT modify this function
    """
    return 1/(1+np.exp(-z))

def softmax(X):
    """
    Do NOT modify this function
    """
    logit = np.exp(X-np.amax(X, axis=1, keepdims=True))
    numer = logit
    denom = np.sum(logit, axis=1, keepdims=True)
    return numer/denom

def load_batch(X, Y, batch_size, shuffle=True):
    """
    Generates batches with the remainder dropped.

    Do NOT modify this function
    """
    if shuffle:
        permutation = np.random.permutation(X.shape[0])
        X = X[permutation, :]
        Y = Y[permutation, :]
    num_steps = int(X.shape[0])//batch_size
    step = 0
    while step<num_steps:
        X_batch = X[batch_size*step:batch_size*(step+1)]
        Y_batch = Y[batch_size*step:batch_size*(step+1)]
        step+=1
        yield X_batch, Y_batch

#2-Layer Neural Network

In [ ]:
class TwoLayerNN:
    """ a neural network with 2 layers """

    def __init__(self, input_dim, num_hiddens, num_classes):
        """
        Do NOT modify this function.
        """
        self.input_dim = input_dim
        self.num_hiddens = num_hiddens
        self.num_classes = num_classes
        self.params = self.initialize_parameters(input_dim, num_hiddens, num_classes)

    def initialize_parameters(self, input_dim, num_hiddens, num_classes):
        """
        initializes parameters with Xavier Initialization.

        Question (a)
        - refer to https://paperswithcode.com/method/xavier-initialization for Xavier initialization

        Inputs
        - input_dim
        - num_hiddens: the number of hidden units
        - num_classes: the number of classes
        Returns
        - params: a dictionary with the initialized parameters.
        """
        params = {}
        params['W1'] = np.random.randn(input_dim, num_hiddens) * np.sqrt(2 / (input_dim + num_hiddens))
        params['b1'] = np.zeros(num_hiddens)
        params['W2'] = np.random.randn(num_hiddens, num_classes) * np.sqrt(2 / (num_hiddens + num_classes))
        params['b2'] = np.zeros(num_classes)
        return params

    def forward(self, X):
        """
        Defines and performs the feed forward step of a two-layer neural network.
        Specifically, the network structue is given by

          y = softmax(sigmoid(X W1 + b1) W2 + b2)

        where X is the input matrix of shape (N, D), y is the class distribution matrix
        of shape (N, C), N is the number of examples (either the entire dataset or
        a mini-batch), D is the feature dimensionality, and C is the number of classes.

        Question (b)
        - ff_dict will be used to run backpropagation in backward method.

        Inputs
        - X: the input matrix of shape (N, D)

        Returns
        - y: the output of the model
        - ff_dict: a dictionary with all the fully connected units and activations.
        """
        W1, b1, W2, b2 = self.params['W1'], self.params['b1'], self.params['W2'], self.params['b2']
        y = softmax(np.dot(sigmoid(np.dot(X,W1) + b1), W2) + b2)
        ff_dict = {'X': X, 'z1': np.dot(X,W1) + b1, 'a1': sigmoid(np.dot(X,W1) + b1), 'z2': np.dot(sigmoid(np.dot(X,W1) + b1), W2) + b2, 'y': y}
        return y, ff_dict

    def backward(self, X, Y, ff_dict):
        """
        Performs backpropagation over the two-layer neural network, and returns
        a dictionary of gradients of all model parameters.

        Question (c)

        Inputs:
         - X: the input matrix of shape (B, D), where B is the number of examples
              in a mini-batch, D is the feature dimensionality.
         - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
              where B is the number of examples in a mini-batch, C is the number
              of classes.
         - ff_dict: the dictionary containing all the fully connected units and
              activations.

        Returns:
         - grads: a dictionary containing the gradients of corresponding weights and biases.
        """
        B = X.shape[0]  # Batch size
        a1, y = ff_dict['a1'], ff_dict['y']

        # Backward pass
        grads = {}
        grads['W2'] = np.dot(a1.T, (y - Y)) / B
        grads['b2'] = np.sum((y - Y), axis=0) / B
        delta2 = np.dot(y - Y, self.params['W2'].T) * a1 * (1 - a1)
        grads['W1'] = np.dot(X.T, delta2) / B
        grads['b1'] = np.sum(delta2, axis=0) / B
        return grads

    def compute_loss(self, Y, Y_hat):
        """
        Computes cross entropy loss.

        Do NOT modify this function.

        Inputs
            Y: ground truth labels
            Y_hat: predicted labels
        Returns
            loss:
        """
        loss = -(1/Y.shape[0]) * np.sum(np.multiply(Y, np.log(Y_hat)))
        return loss

    def train(self, X, Y, X_val, Y_val, lr, n_epochs, batch_size, log_interval=1):
        """
        Runs mini-batch gradient descent.

        Do NOT Modify this method.

        Inputs
        - X
        - Y
        - X_val: validation data
        - Y_Val: validation label
        - lr: learning rate
        - n_epochs: the number of epochs to run
        - batch_size
        - log_interval: the epoch interval to log the training progress.
        """
        for epoch in range(n_epochs):
            for X_batch, Y_batch in load_batch(X, Y, batch_size):
                self.train_step(X_batch, Y_batch, batch_size, lr)
            if epoch % log_interval==0:
                Y_hat, ff_dict = self.forward(X)
                train_loss = self.compute_loss(Y, Y_hat)
                train_acc = self.evaluate(Y, Y_hat)
                Y_hat, ff_dict = self.forward(X_val)
                valid_loss = self.compute_loss(Y_val, Y_hat)
                valid_acc = self.evaluate(Y_val, Y_hat)
                print('epoch {:02} - train loss/acc: {:.3f} {:.3f}, valid loss/acc: {:.3f} {:.3f}'.\
                      format(epoch, train_loss, train_acc, valid_loss, valid_acc))

    def train_step(self, X_batch, Y_batch, batch_size, lr):
        """
        Updates the parameters using gradient descent.

        Question (d)

        Inputs
        - X_batch
        - Y_batch
        - batch_size
        - lr: learning rate
        """
        _, ff_dict = self.forward(X_batch)
        grads = self.backward(X_batch, Y_batch, ff_dict)

        self.params['W1'] -= lr * grads['W1']
        self.params['b1'] -= lr * grads['b1']
        self.params['W2'] -= lr * grads['W2']
        self.params['b2'] -= lr * grads['b2']

    def evaluate(self, Y, Y_hat):
        """
        Computes classification accuracy.

        Question (e)

        Inputs
        - Y: A numpy array of shape (N, C) containing the one-hot encoded labels,
             where C is the number of classes.
        - Y_hat: A numpy array of shape (N, C) containing the softmax outputs,
             where C is the number of classes.

        Returns
            accuracy: the classification accuracy in float
        """
        predicted_labels = np.argmax(Y_hat, axis=1)
        true_labels = np.argmax(Y, axis=1)
        accuracy = np.mean(predicted_labels == true_labels)

        return accuracy

#Load MNIST

In [ ]:
X_train, Y_train, X_test, Y_test = load_data()

idxs = np.arange(len(X_train))
np.random.shuffle(idxs)
split_idx = int(np.ceil(len(idxs)*0.8))
X_valid, Y_valid = X_train[idxs[split_idx:]], Y_train[idxs[split_idx:]]
X_train, Y_train = X_train[idxs[:split_idx]], Y_train[idxs[:split_idx]]
print()
print('Set validation data aside')
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', Y_train.shape)
print('Validation data shape: ', X_valid.shape)
print('Validation labels shape: ', Y_valid.shape)

MNIST data loaded:
Training data shape: (60000, 784)
Training labels shape: (60000, 10)
Test data shape: (10000, 784)
Test labels shape: (10000, 10)

Set validation data aside
Training data shape:  (48000, 784)
Training labels shape:  (48000, 10)
Validation data shape:  (12000, 784)
Validation labels shape:  (12000, 10)


#Training & Evaluation

In [ ]:
###
# Question (f)
# Tune the hyperparameters with validation data,
# and print the results by running the lines below.
###

In [ ]:
# model instantiation
model = TwoLayerNN(input_dim=784, num_hiddens=10, num_classes=10)

In [ ]:
# train the model & evaluate with validation data
lr, n_epochs, batch_size = 0.1, 100, 32
model.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

epoch 00 - train loss/acc: 0.521 0.874, valid loss/acc: 0.527 0.869
epoch 01 - train loss/acc: 0.374 0.902, valid loss/acc: 0.382 0.896
epoch 02 - train loss/acc: 0.328 0.911, valid loss/acc: 0.339 0.906
epoch 03 - train loss/acc: 0.305 0.915, valid loss/acc: 0.321 0.908
epoch 04 - train loss/acc: 0.289 0.919, valid loss/acc: 0.307 0.911
epoch 05 - train loss/acc: 0.274 0.923, valid loss/acc: 0.296 0.913
epoch 06 - train loss/acc: 0.268 0.924, valid loss/acc: 0.291 0.915
epoch 07 - train loss/acc: 0.258 0.926, valid loss/acc: 0.285 0.918
epoch 08 - train loss/acc: 0.254 0.927, valid loss/acc: 0.284 0.917
epoch 09 - train loss/acc: 0.249 0.928, valid loss/acc: 0.279 0.918
epoch 10 - train loss/acc: 0.244 0.931, valid loss/acc: 0.276 0.919
epoch 11 - train loss/acc: 0.241 0.931, valid loss/acc: 0.278 0.919
epoch 12 - train loss/acc: 0.236 0.933, valid loss/acc: 0.275 0.920
epoch 13 - train loss/acc: 0.233 0.933, valid loss/acc: 0.273 0.921
epoch 14 - train loss/acc: 0.227 0.935, valid lo

KeyboardInterrupt: 

In [ ]:
# evalute the model on test data
Y_hat, _ = model.forward(X_test)
test_loss = model.compute_loss(Y_test, Y_hat)
test_acc = model.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))

Final test loss = 2.356, acc = 0.114


# Extra Credit (Optional)

In [ ]:
def initialize_parameters(self, input_dim, num_hiddens, num_classes):
    """
    initializes parameters with He Initialization.

    Question (g)
    - refer to https://paperswithcode.com/method/he-initialization for He initialization

    Inputs
    - input_dim
    - num_hiddens
    - num_classes
    Returns
    - params: a dictionary with the initialized parameters.
    """

def forward_leaky_relu(self, X):
    """
    Defines and performs the feed forward step of a two-layer neural network.
    Specifically, the network structue is given by

        y = softmax(leaky_relu(X W1 + b1) W2 + b2)

    where X is the input matrix of shape (N, D), y is the class distribution matrix
    of shape (N, C), N is the number of examples (either the entire dataset or
    a mini-batch), D is the feature dimensionality, and C is the number of classes.
    The gradient of the leaky relu activation is 1 for x > 0 and 0.01 for x <= 0.

    Question (g)

    Inputs
        X: the input matrix of shape (N, D)

    Returns
        y: the output of the model
        ff_dict: a dictionary containing all the fully connected units and activations.
    """

def backward_leaky_relu(self, X, Y, ff_dict):
    """
    Performs backpropagation over the two-layer neural network, and returns
    a dictionary of gradients of all model parameters.

    Question (g)

    Inputs:
        - X: the input matrix of shape (B, D), where B is the number of examples
            in a mini-batch, D is the feature dimensionality.
        - Y: the matrix of one-hot encoded ground truth classes of shape (B, C),
            where B is the number of examples in a mini-batch, C is the number
            of classes.
        - ff_dict: the dictionary containing all the fully connected units and
            activations.

    Returns:
        - grads: a dictionary containing the gradients of corresponding weights
            and biases.
    """

TwoLayerNNLeakyRelu = copy.copy(TwoLayerNN)
TwoLayerNNLeakyRelu.initialize_parameters = initialize_parameters
TwoLayerNNLeakyRelu.feed_forward = forward_leaky_relu
TwoLayerNNLeakyRelu.back_propagate = backward_leaky_relu

In [ ]:
###
# Question (g)
# Tune the hyperparameters with validation data,
# and print the results by running the lines below.
###

In [ ]:
# model instantiation
model_leaky_relu = TwoLayerNNLeakyRelu(input_dim=0, num_hiddens=0, num_classes=0)

In [ ]:
# train the model & evaluate with validation data
lr, n_epochs, batch_size = 0, 0, 0
history = model_leaky_relu.train(X_train, Y_train, X_valid, Y_valid, lr, n_epochs, batch_size)

In [ ]:
Y_hat, _ = model_leaky_relu.forward(X_test)
test_loss = model_leaky_relu.compute_loss(Y_test, Y_hat)
test_acc = model_leaky_relu.evaluate(Y_test, Y_hat)
print("Final test loss = {:.3f}, acc = {:.3f}".format(test_loss, test_acc))